In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [32]:
df=pd.read_csv("D:/Semester5Project/og.csv",low_memory=False)

In [33]:
df.head()

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
1,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
2,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
3,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
4,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,NaN,0


In [34]:
df.isnull().any()

srcip               False
sport               False
dstip               False
dsport              False
proto               False
state               False
dur                 False
sbytes              False
dbytes              False
sttl                False
dttl                False
sloss               False
dloss               False
service             False
Sload               False
Dload               False
Spkts               False
Dpkts               False
swin                False
dwin                False
stcpb               False
dtcpb               False
smeansz             False
dmeansz             False
trans_depth         False
res_bdy_len         False
Sjit                False
Djit                False
Stime               False
Ltime               False
Sintpkt             False
Dintpkt             False
tcprtt              False
synack              False
ackdat              False
is_sm_ips_ports     False
ct_state_ttl        False
ct_flw_http_mthd     True
is_ftp_login

In [35]:
df.drop(index=df[df.duplicated()].index,inplace=True)

In [22]:
for i in df.select_dtypes(exclude=np.number).columns:
    print(i,df[i].str.isspace().any())

srcip False
sport False
dstip False
dsport False
proto False
state False
service False
ct_ftp_cmd True
attack_cat False


In [36]:
df['ct_ftp_cmd'].value_counts()

     1014165
0    1005477
1      38853
2        263
4        250
3        206
5        105
6         91
8          3
Name: ct_ftp_cmd, dtype: int64

In [37]:
indexes=[]
for i,j in df['ct_ftp_cmd'].str.isspace().items():
    if(j==True):
        indexes.append(i)
df['ct_ftp_cmd'].loc[indexes]='0'

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [38]:
df['ct_ftp_cmd'].value_counts()

0    2019642
1      38853
2        263
4        250
3        206
5        105
6         91
8          3
Name: ct_ftp_cmd, dtype: int64

Dropping the IP Addresses as they are not useful (and attack_cat)

In [39]:
df.drop(columns=['srcip','dstip','attack_cat'],inplace=True)

In [43]:
df=df.replace('-',np.nan)

In [46]:
df['sport']=df['sport'].apply(lambda x: '12' if x=='0x000c' else x)
df['sport']=df['sport'].apply(lambda x: '11' if x=='0x000b' else x)
df['sport']=df['sport'].astype('float64')

In [47]:
df['dsport']=df['dsport'].apply(lambda x: '52233' if x=='0xcc09' else x)
df['dsport']=df['dsport'].apply(lambda x: '49320' if x=='0xc0a8' else x)
df['dsport']=df['dsport'].apply(lambda x: np.nan if x=='0x20205321' else x)
df['dsport']=df['dsport'].astype('float64')

In [49]:
df['ct_ftp_cmd']=df['ct_ftp_cmd'].astype(int)

In [50]:
df.select_dtypes(exclude=np.number)

,proto,state,service
0,udp,CON,NaN
1,udp,CON,dns
2,udp,CON,dns
3,udp,CON,dns
4,udp,CON,NaN
...,...,...,...
2540035,tcp,CON,http
2540037,tcp,CON,NaN
2540038,tcp,FIN,ftp-data
2540041,tcp,CON,http


In [57]:
dummies=pd.get_dummies(df[['proto','state','service']])

In [58]:
df.drop(columns=['proto','state','service'],inplace=True)

In [64]:
scaler=MinMaxScaler()
df.iloc[:,0:42]=scaler.fit_transform(df.iloc[:,0:42])

In [65]:
df.head()

,sport,dsport,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,...,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
0,0.513634,0.015625,4.112267e-06,0.000037,0.000021,0.121569,0.114173,0.0,0.0,0.000015,...,0.0,0.0,0.015152,0.045455,0.015152,0.030303,0.000000,0.0,0.015152,0
1,0.022339,0.000809,1.273525e-07,0.000010,0.000012,0.121569,0.114173,0.0,0.0,0.000087,...,0.0,0.0,0.166667,0.106061,0.000000,0.015152,0.015152,0.0,0.000000,0
2,0.054826,0.000809,1.375953e-07,0.000009,0.000011,0.121569,0.114173,0.0,0.0,0.000073,...,0.0,0.0,0.075758,0.121212,0.000000,0.000000,0.000000,0.0,0.000000,0
3,0.757824,0.000809,1.330429e-07,0.000010,0.000012,0.121569,0.114173,0.0,0.0,0.000083,...,0.0,0.0,0.090909,0.121212,0.000000,0.000000,0.000000,0.0,0.000000,0
4,0.490105,0.001694,8.915697e-06,0.000040,0.000021,0.121569,0.114173,0.0,0.0,0.000007,...,0.0,0.0,0.015152,0.045455,0.015152,0.030303,0.000000,0.0,0.015152,0


In [66]:
df.shape

(2059413, 43)

In [67]:
df=pd.concat([df,dummies],axis=1)
df.shape

(2059413, 206)

In [68]:
df.to_csv('D:/Semester5Project/datasetnorm.csv',chunksize=512)